In [2]:
import pprint
from ollama import chat
import json
from os import listdir
from os.path import join
import re

In [3]:
# def compile_SYS_PROMT():
# 	return """
# 	You are a social innovator, helping me write a glossary on tools, methods, tactics, principles and skills 
#     that are commonly used in social and public innovation work.

#     I will share with you a COMPILATION OF NOTES in markdown format that you will summarize the content for a single entry in the glossary.
#     You return only the summary in a structured markdown format.
# 	"""

In [31]:
def compile_SYS_PROMT():
	return """
	You are a social innovator, helping me write a glossary on tools, methods, tactics, principles and skills 
    that are commonly used in public and social innovation work. The audience of this glossary are public officials
    and civil servants who have some understanding of public and social innovation, but are not experts.

    I will share with you a COMPILATION OF NOTES in markdown format 
    that you will summarize for a single entry in the glossary.
    You will return only the summary in a structured markdown format, using the sections provided below between three back ticks. 
    Only fill out the sections in that structure. Do not create a new structure, new sections, or follow 
    the structure of the COMPILATION OF NOTES.
    Do not add an introductory sentence to your response.

    Markdown structure:
    ```
    ## Overview
    ## Purpose
    ## How it works
    ## Requirements
    ## Reflections
    ## Related methods (if any)
    ## License
    ## Examples
    ```
	"""

In [32]:
def summarize_search_result(**kwargs):
	prompt = kwargs.get('prompt')
	md = kwargs.get('md')
	stream_output = kwargs.get('stream_output', False)
	model = kwargs.get('model', 'llama3.1:8b')

	if prompt is None:
		raise Exception('missing prompt')
	if md is None:
		raise Exception('missing json input')

	stream: ChatResponse = chat(
		model=model, 
		options={
			'seed': 42,
			'temperature': 0.2,
			'num_ctx': 20000,
		},
		messages = [
			{
				'role': 'system',
				'content': compile_SYS_PROMT(),
			},
			{
				'role': 'user',
				'content': f"""
					COMPILATION OF NOTES: {md}
					USER PROMPT: {prompt}
				"""
			},
		],
		stream=stream_output,
	)

	full_response = ''

	if stream_output == True:
		print('[AGENT] SUMMARY:')
		for chunk in stream:
			print(chunk['message']['content'], end='', flush=True)
			full_response += chunk['message']['content']
		print('\n')
	else:
		full_response = stream['message']['content']

	return full_response

In [33]:
%run load_docx.ipynb

In [34]:
directory = f'../elements/__pages__/'
files = [f for f in listdir(directory) if f != '.DS_Store']

In [35]:
def getTypes (md):
    types = re.findall(r'(?<=\[\[type:)[\w:.\-\/\d\s…&\(\)\",]*(?=\]\])', str(md))
    types = [t[:-1] for t in types if t[-1] == 's']
    return types

In [36]:
def getElementName (md):
    # print(md.decode('unicode_escape'))
    # print(md)
    element = re.findall(r'(?<=\n# )[\w\'"’”‘“\(\)\[\]_\-\s]+(?=\n)', str(md), flags=re.MULTILINE)
    element = [e.strip() for e in element]
    return element[0]

In [37]:
for f in files[:1]:
    # f = 'Accessibility of knowledge.md'
    md = readFile(join(directory, f))
    element = getElementName(md)
    print(element)
    element_type = ' or '.join(getTypes(md))
    print(element_type)
    prompt = f'I need to write an entry on {element} as a {element_type} for a social innovation glossary'
    # Summarize
    content = summarize_search_result(prompt=prompt, md=md, stream_output=False)
    print(content)
    # Export summary
    # output_file = open(join('../elements/__generated__', f), 'w')
    # output_file.write(content)
    # output_file.close()

Nesta’s DIY Toolkit
tool
Here's a draft of the entry:

**DIY Toolkit**

The DIY Toolkit is a collection of 30 practical social innovation tools, designed to help development practitioners invent, adopt or adapt ideas that can deliver better results. The toolkit features canvases and questions, illustrated with use-cases, that are quick to use and simple to apply.

**Purpose:**
The DIY Toolkit has been created to support the development community in finding innovative solutions to complex problems. It is designed for use by practitioners working on social innovation projects, particularly those focused on sustainable development, human rights, and social justice.

**How it works:**

1. Download the toolkit from the Nesta website.
2. Choose a canvas or question that aligns with your project's needs.
3. Use the canvas or question to guide your work, either individually or in collaboration with others.

**Key features:**

* 30 practical social innovation tools
* Canvases and questions for 